## HTTP server

### Description

When contacted by the DHCP client, the DHCP server returns an HTTP URL together with the IP address assigned to the IOS-XR. The ZTP process then contacts the HTTP server and requests the content of its configuration file. On its turn, the HTTP server will call an action on NSO to retrieve the day0 configuration file based on the device serial. After the retrieval, the ZTP process applies the received day-0 configuration and finishes with a notification sent back to the HTTP server that announces its readiness for onboarding. The HTTP server then receives this information and triggers the device onboarding process in NSO.

The implementation of this HTTP server is written in Python v3 with Flask v1.1.1 as a web framework and ncclient as a NETCONF client library. The application runs on the NSO host, available on all interfaces on port 8181 (198.18.134.50:8181). The server listens to incoming HTTP requests for the two phases of the ZTP process:

•	Retrieval of day-0 configuration for the device

•	Readiness for onboarding of the device in NSO


### Necessary imports and variable definitiion


In [1]:
import os
import sys
import shutil
import atexit
import logging

from ncclient import manager
from xml.etree import ElementTree
from ncclient.xml_ import to_ele
from ncclient.operations import RPCError

from flask import Flask, send_from_directory

app = Flask(__name__)
temporary_dir = 'tmp'
session = {}


### Initial configuration

Set the credentials used to connect to NSO over netconf and
load the 2 different xml payload which will be sent to nso, corresponding to the 2 defined actions - day0 retrieval and device onboarding.

In [2]:
def initial_config():

    # NSO credentials
    session['nso'] = {}
    session['nso']['host'] = "198.18.134.50"
    session['nso']['port'] = "2022"
    session['nso']['user'] = "admin"
    session['nso']['pass'] = "admin"

    # Read NSO RPC action for request of configuration file
    file_name = "config/config-action.xml"
    with open(file_name, 'r') as file:
        session['nso']['config-action'] = file.read()

    # Read NSO RPC action for request of onboarding
    file_name = "config/onboard-action.xml"
    with open(file_name, 'r') as file:
        session['nso']['onboard-action'] = file.read()

    # Create temporary directory that stores config files
    if not os.path.exists(temporary_dir):
        os.mkdir(temporary_dir)
        

### Get day0 configuration method

We define a GET/HEAD route (url) on the server to return the configuration file by calling an NSO action which, based on the device serial, will provide the contents of the correspoinding day0 file.
The method will be called whenever there is an incoming HEAD or GET request of this form:

http://198.18.134.50:8181/ztp/conf/v1/{serial}
    
where {serial} should be the serial of a device which is already preconfigured in the NSO ztp service.

For communicating with NSO we use the ncclient library to connect to the NETCONF interface. 

The payload which we send using the ncclient library to call the get_day0 action from the ztp service is:
```xml
<action xmlns="http://tail-f.com/ns/netconf/actions/1.0">
  <data>
    <ztp xmlns="http://example.com/ztp">
      <serial>{SERIAL}</serial>
      <get_day0/>
    </ztp>
  </data>
</action>
```

where {SERIAL} will be replaced with the serial number coming from the device (included in the URL)

    

In [3]:
@app.route('/ztp/conf/v1/<serial>')
def conf_v1(serial):
    m = manager.connect(look_for_keys=False,
                        host=session['nso']['host'],
                        port=session['nso']['port'],
                        username=session['nso']['user'],
                        password=session['nso']['pass'],
                        hostkey_verify=False)

    try:
        # Call get config action
        if session['nso']['config-action'] is not None and len(session['nso']['config-action']) > 0:
            try:
                response = m.dispatch(to_ele(session['nso']['config-action'].format(SERIAL=serial)))
                configuration = response.data_ele.find(".//*{http://example.com/ztp}message")

                # Return a configuration file
                if configuration is not None:
                    with open(os.path.join(temporary_dir, serial), 'w') as file:
                        file.write(configuration.text)

                    return send_from_directory(temporary_dir, serial,
                                               as_attachment=True)
            except RPCError as e:
                print('Failed to make RPC request:', e)
    finally:
        m.close_session()

    return ''


### Onboard device in NSO method

In order know when the device finished applying the day0 configuration we add as the last step in the config script (bash, python) a POST call to our HTTP server. The URL to be called will look like:

http://198.18.134.50:8181/ztp/status/{serial}/1

(the last parameter is the status code - for now only status 1 is used which means OK, but additional statuses might be used to have a more detailed feedback during the execution of the script)

Upon retrieving this request, the HTTP server will call the onboard action on NSO - again by specifiying the serial of the devidce which is part of the URL. 

```xml
<action xmlns="http://tail-f.com/ns/netconf/actions/1.0">
  <data>
    <ztp xmlns="http://example.com/ztp">
      <serial>{SERIAL}</serial>
      <onboard/>
    </ztp>
  </data>
</action>
```

NSO in its turn will onboard the device - add it to the device tree, fetch the ssh keys and perform a sync-from.


In [4]:
@app.route('/ztp/status/<serial>/<int:status>', methods=['POST'])
def onboard(serial, status):

    m = manager.connect(look_for_keys=False,
                        host=session['nso']['host'],
                        port=session['nso']['port'],
                        username=session['nso']['user'],
                        password=session['nso']['pass'],
                        hostkey_verify=False)
    
    try:
        if serial and status == 1:
            # Call onboarding action
            if session['nso']['onboard-action'] is not None and len(session['nso']['onboard-action']) > 0:
                try:
                    m.dispatch(to_ele(session['nso']['onboard-action'].format(SERIAL=serial)))
                except RPCError as e:
                    print('Failed to make RPC request:', e)

    finally:
        m.close_session()

    return ''



### Start the web server and wait for requests


In [ ]:
if __name__ == '__main__':

    initial_config()
    # Expose server through all IP addresses, on port 8181
    app.run(host='0.0.0.0', port=8181)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:8181/ (Press CTRL+C to quit)
198.18.1.32 - - [28/Jan/2020 07:46:10] "HEAD /ztp/conf/v1/36e6df2c HTTP/1.1" 200 -
198.18.1.32 - - [28/Jan/2020 07:46:11] "GET /ztp/conf/v1/36e6df2c HTTP/1.1" 200 -
198.18.1.32 - - [28/Jan/2020 07:46:23] "POST /ztp/status/36e6df2c/1 HTTP/1.1" 200 -
198.18.1.32 - - [28/Jan/2020 07:48:23] "HEAD /ztp/conf/v1/36e6df2c HTTP/1.1" 200 -
198.18.1.32 - - [28/Jan/2020 07:48:24] "GET /ztp/conf/v1/36e6df2c HTTP/1.1" 200 -
198.18.1.32 - - [28/Jan/2020 07:48:36] "POST /ztp/status/36e6df2c/1 HTTP/1.1" 200 -
198.18.1.32 - - [28/Jan/2020 07:51:21] "HEAD /ztp/conf/v1/36e6df2c HTTP/1.1" 200 -
198.18.1.32 - - [28/Jan/2020 07:51:21] "GET /ztp/conf/v1/36e6df2c HTTP/1.1" 200 -
198.18.1.32 - - [28/Jan/2020 07:51:34] "POST /ztp/status/36e6df2c/1 HTTP/1.1" 200 -


### Verify that the request for getting the day0 configuration file is working:

#### TASK

Click on this link: 
http://198.18.134.50:8181/ztp/conf/v1/36e6df2c

and check that the day0 configuration is downloaded with the placeholders replaced with the saved values for our devices
